In [1]:
%matplotlib inline
from pycbc import filter
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector
import pylab
from pycbc.waveform import td_approximants, fd_approximants

import os
import h5py
import numpy as np
import pandas as pd
import seaborn as sns


from gwpy.time import to_gps, from_gps
from gwpy.table import Table
from gwpy.time import tconvert
from gwdetchar import scattering
from gwpy.table import EventTable
from gwpy.segments import Segment
from gwpy.segments import SegmentList
from gwpy.timeseries import TimeSeries
from gwtrigfind import find_trigger_files
from gwpy.segments import DataQualityDict
from gwpy.segments import DataQualityFlag
from gwpy.timeseries import TimeSeriesDict
from gwpy.spectrogram import Spectrogram

import glob
import time
import datetime
from datetime import timedelta
from itertools import islice
from scipy.signal import correlate
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

In [2]:
def gen_thresholds(dataframe, thresholds = [2,1,1]):
    
    df = dataframe
    if len(thresholds) < 4:
        thresh_cond = (df['t1'] > thresholds[0]) | (df['t2'] > thresholds[1]) | (df['t3'] > thresholds[2]) 
    else:
        thresh_cond = (df['t1']>thresholds[0]) | (df['t2']>thresholds[1]) |(df['t3']>thresholds[2]) | (df['t4']>thresholds[3]) | (df['t5'] > thresholds[4])
        
    return thresh_cond

### Injections and background, overlap and non overlap analysis

In [4]:
df_inj_ovlp = pd.read_csv('files/combined_bbh_new.csv')
df_inj_nonovlp = pd.read_csv('files/combined_bbh_mutexc.csv')

In [5]:
df_bkg_ovlp = pd.read_csv('files/combined_bkg_ovlp.csv')
df_bkg_nonovlp = pd.read_csv('files/combined_bkg_nonovlp.csv')

In [8]:
df_bkg_nonovlp['filename'].nunique()

1591

#### The files df_inj_ovlp, df_inj_nonovlp, df_bkg_ovlp and df_bkg_nonovlp contain overlap and non overlap analysis with energy threshold of 60 for the 1354 injections and 1591 background events
The injection files on which the calc_pixel_occupancy was ran to produce df_inj_ovlp and df_inj_nonovlp are stored at '/home/olib/public_html/Ethan/O3_replay_inj/0lag_lowfreq/q_transforms/BBH_data_snr_5/' and the background files on which the calc_pixel_occupancy was ran to produce df_bkg_ovlp and df_bkg_nonovlp is stored at '/home/ethan.marx/forks/retraction_scripts/projects/data/background/raw/'

### Attributes of injections and background

In [6]:
df_inj_attr = pd.read_csv('files/combined_bbh_new_attrs.csv')

In [7]:
df_bkg_attr = pd.read_csv('files/combined_bkg_attrs_nonovlp.csv')

### The dataframes df_inj_attr and df_bkg_attr contain attributes of the 1354 injections and 1591 background events. 

In [9]:
df_inj_attr

,replay_time,time_H,coa_phase,eff_dist_H,approximant,chi_p,z,distance,spin2z,fref,...,Mc,lambda1,mass2,flow,logpdraw_right_ascension_declination,logpdraw_z_mass1_source_mass2_source_spin1x_spin1y_spin1z_spin2x_spin2y_spin2z,logpdraw_lambda1_lambda2_GIVEN_mass1_source_mass2_source,logpdraw_inclination_polarization_coa_phase,logpdraw_eccentricity,logpdraw_time
0,1.324070e+09,1.263782e+09,4.400568,658.213588,b'IMRPhenomPv2_NRTidalv2',0.489192,0.205563,1048.320953,-0.083213,10.0,...,12.682835,0.0,13.432107,10.0,-2.824640,-11.318613,0.0,-4.490021,0.0,-inf
1,1.322883e+09,1.262595e+09,5.545442,512.341438,b'IMRPhenomPv2_NRTidalv2',0.552983,0.159667,792.627414,-0.099114,10.0,...,13.765229,0.0,13.922637,10.0,-3.313658,-12.801971,0.0,-3.979053,0.0,-inf
2,1.323322e+09,1.263034e+09,2.085905,1146.963429,b'IMRPhenomPv2_NRTidalv2',0.074477,0.323853,1757.498679,-0.011599,10.0,...,27.222245,0.0,30.848315,10.0,-3.504506,-2.906306,0.0,-4.459404,0.0,-inf
3,1.325999e+09,1.265711e+09,2.555068,439.029648,b'IMRPhenomPv2_NRTidalv2',0.222893,0.201676,1026.221976,0.013461,10.0,...,91.721406,0.0,102.310778,10.0,-2.591834,-9.378661,0.0,-4.334820,0.0,-inf
4,1.324281e+09,1.263993e+09,6.249260,861.692729,b'IMRPhenomPv2_NRTidalv2',0.285242,0.202187,1029.121154,-0.712433,10.0,...,35.023265,0.0,29.641763,10.0,-2.930356,-12.596220,0.0,-4.518973,0.0,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,1.324834e+09,1.264546e+09,3.709780,542.083678,b'IMRPhenomPv2_NRTidalv2',0.345502,0.295165,1579.237687,-0.216031,10.0,...,38.210390,0.0,36.210617,10.0,-2.675384,-11.734877,0.0,-3.750769,0.0,-inf
1350,1.323042e+09,1.262754e+09,2.631282,1004.274519,b'IMRPhenomPv2_NRTidalv2',0.573371,0.372609,2069.033503,-0.083389,10.0,...,87.795678,0.0,84.562523,10.0,-3.033623,-14.318081,0.0,-3.785242,0.0,-inf
1351,1.324072e+09,1.263784e+09,3.237331,889.423392,b'IMRPhenomPv2_NRTidalv2',0.531267,0.324816,1763.546735,-0.187699,10.0,...,59.498874,0.0,50.079667,10.0,-2.660177,-14.794976,0.0,-4.699917,0.0,-inf
1352,1.323139e+09,1.262851e+09,0.769810,849.112932,b'IMRPhenomPv2_NRTidalv2',0.721460,0.325681,1768.988754,-0.360885,10.0,...,73.213026,0.0,73.168037,10.0,-2.834519,-15.555550,0.0,-4.704570,0.0,-inf


In [10]:
df_bkg_attr

,ifo_freq,fmin,fres,ifar,m1,m2,time,tres,window,filename
0,0,10.0,0.05,70.549339,18.581810,13.023375,1.248149e+09,0.01,2.0,acb180ce-4845-4652-9139-6752941fc844
1,0,10.0,0.05,79.977180,9.023204,5.538040,1.247408e+09,0.01,2.0,3c99d9bc-5103-4c52-8a49-c31e4114e8a5
2,0,10.0,0.05,77.122160,15.986259,7.134080,1.252482e+09,0.01,2.0,8ba4bd72-8d07-4986-925d-229e20431c04
3,0,10.0,0.05,416.761820,20.475435,7.730863,1.247266e+09,0.01,2.0,ee9849cf-91a5-45e6-adef-e1ffa556a72f
4,0,10.0,0.05,68.722388,489.172450,9.301118,1.238307e+09,0.01,2.0,16df310e-9972-4b2a-be1a-a67338b3accd
...,...,...,...,...,...,...,...,...,...,...
1586,0,10.0,0.05,79.632953,46.586437,9.287184,1.238176e+09,0.01,2.0,c579ddb2-3487-4f5a-aad0-76496e2d7be0
1587,0,10.0,0.05,165.747140,66.721344,5.479237,1.246106e+09,0.01,2.0,0f6e6114-0da9-41ca-a1cb-fbdd7757efe5
1588,0,10.0,0.05,83.958693,7.565491,5.176327,1.241144e+09,0.01,2.0,b5719a8b-4143-4ef7-9c03-8dcd316c56c5
1589,0,10.0,0.05,87.723144,492.076930,5.817298,1.252567e+09,0.01,2.0,7f5e61ea-cd1e-4572-ac73-cfae0ea2519d


### Let's now look at injections flagged first by non overlap method and then by overlap method

In [11]:
df_inj_nonovlp_flag_a = df_inj_nonovlp[gen_thresholds(df_inj_nonovlp, thresholds=[0,0,4,0,0])]

In [12]:
df_inj_nonovlp_flag_a.head()

,ifo_freq,t1,t2,t3,t4,t5,time
12,L1_f1,0.0,0.0,0.3,0.1,5.0,1325566299
13,L1_f2,0.0,0.0,0.0,0.2,0.0,1325566299
30,L1_f1,0.0,0.4,1.2,0.0,0.5,1325410502
408,L1_f1,2.1,0.0,0.0,1.2,2.3,1322690262
411,H1_f1,0.0,1.0,0.3,0.0,2.1,1322690262


df_inj_nonovlp_flag_a are events flagged by non overlap method with the thresholds [0,0,4,0,0]

In [13]:
df_inj_nonovlp_flag_a_inovlp = df_inj_ovlp[df_inj_ovlp['time'].isin(df_inj_nonovlp_flag_a['time'])]

In [14]:
df_inj_nonovlp_flag_a_inovlp

,ifo_freq,t1,t2,t3,time
198,L1_f1,5.4,2.7,1.3,1324331465
199,L1_f2,0.7,0.3,0.2,1324331465
200,L1_f3,0.0,0.0,0.0,1324331465
201,H1_f1,1.8,0.9,0.5,1324331465
202,H1_f2,0.5,0.2,0.1,1324331465
...,...,...,...,...,...
8065,L1_f2,0.0,0.0,0.0,1325865224
8066,L1_f3,0.0,0.0,0.0,1325865224
8067,H1_f1,0.0,0.0,0.0,1325865224
8068,H1_f2,0.0,0.0,0.0,1325865224


df_inj_nonovlp_flag_a_inovlp is the part of df_inj_ovlp which is in df_inj_nonovlp_flag_a, we will now apply the overlap threhsolds on this to find events flagged by both

In [15]:
df_inj_both_flag_a = df_inj_nonovlp_flag_a_inovlp[gen_thresholds(df_inj_nonovlp_flag_a_inovlp, thresholds=[4,2,2])]

In [16]:
df_inj_both_flag_a.head()

,ifo_freq,t1,t2,t3,time
198,L1_f1,5.4,2.7,1.3,1324331465
306,L1_f1,3.6,2.3,1.1,1325581835
330,L1_f1,5.6,2.8,1.4,1322914217
936,L1_f1,4.2,2.1,1.0,1324788826
1332,L1_f1,8.9,4.7,2.3,1325614061


The information in the dataframe df_inj_both_flag_a is also stored in the csv file /home/siddharth.soni/Retraction_project/Retraction_paper/inj_abvnonvlp_inovlp.csv

df_inj_flagged_both_a are the events that are flagged by both, first by nonoverlap and then by overlap method with thresholds ([0,0,4,0,0] and [4,2,2]) respectively, energy threshold 60

In [37]:
#df_inj_flagged_both_a = pd.read_csv('/home/siddharth.soni/Retraction_project/Retraction_paper/inj_abvnonvlp_inovlp.csv')

### Injection attributes

In [17]:
df_inj_attr

,replay_time,time_H,coa_phase,eff_dist_H,approximant,chi_p,z,distance,spin2z,fref,...,Mc,lambda1,mass2,flow,logpdraw_right_ascension_declination,logpdraw_z_mass1_source_mass2_source_spin1x_spin1y_spin1z_spin2x_spin2y_spin2z,logpdraw_lambda1_lambda2_GIVEN_mass1_source_mass2_source,logpdraw_inclination_polarization_coa_phase,logpdraw_eccentricity,logpdraw_time
0,1.324070e+09,1.263782e+09,4.400568,658.213588,b'IMRPhenomPv2_NRTidalv2',0.489192,0.205563,1048.320953,-0.083213,10.0,...,12.682835,0.0,13.432107,10.0,-2.824640,-11.318613,0.0,-4.490021,0.0,-inf
1,1.322883e+09,1.262595e+09,5.545442,512.341438,b'IMRPhenomPv2_NRTidalv2',0.552983,0.159667,792.627414,-0.099114,10.0,...,13.765229,0.0,13.922637,10.0,-3.313658,-12.801971,0.0,-3.979053,0.0,-inf
2,1.323322e+09,1.263034e+09,2.085905,1146.963429,b'IMRPhenomPv2_NRTidalv2',0.074477,0.323853,1757.498679,-0.011599,10.0,...,27.222245,0.0,30.848315,10.0,-3.504506,-2.906306,0.0,-4.459404,0.0,-inf
3,1.325999e+09,1.265711e+09,2.555068,439.029648,b'IMRPhenomPv2_NRTidalv2',0.222893,0.201676,1026.221976,0.013461,10.0,...,91.721406,0.0,102.310778,10.0,-2.591834,-9.378661,0.0,-4.334820,0.0,-inf
4,1.324281e+09,1.263993e+09,6.249260,861.692729,b'IMRPhenomPv2_NRTidalv2',0.285242,0.202187,1029.121154,-0.712433,10.0,...,35.023265,0.0,29.641763,10.0,-2.930356,-12.596220,0.0,-4.518973,0.0,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,1.324834e+09,1.264546e+09,3.709780,542.083678,b'IMRPhenomPv2_NRTidalv2',0.345502,0.295165,1579.237687,-0.216031,10.0,...,38.210390,0.0,36.210617,10.0,-2.675384,-11.734877,0.0,-3.750769,0.0,-inf
1350,1.323042e+09,1.262754e+09,2.631282,1004.274519,b'IMRPhenomPv2_NRTidalv2',0.573371,0.372609,2069.033503,-0.083389,10.0,...,87.795678,0.0,84.562523,10.0,-3.033623,-14.318081,0.0,-3.785242,0.0,-inf
1351,1.324072e+09,1.263784e+09,3.237331,889.423392,b'IMRPhenomPv2_NRTidalv2',0.531267,0.324816,1763.546735,-0.187699,10.0,...,59.498874,0.0,50.079667,10.0,-2.660177,-14.794976,0.0,-4.699917,0.0,-inf
1352,1.323139e+09,1.262851e+09,0.769810,849.112932,b'IMRPhenomPv2_NRTidalv2',0.721460,0.325681,1768.988754,-0.360885,10.0,...,73.213026,0.0,73.168037,10.0,-2.834519,-15.555550,0.0,-4.704570,0.0,-inf


### Let's now look at the attributes of injections flagged by both non overlap and overlap methods

In [18]:
df_inj_both_flag_a_attrs = df_inj_attr[df_inj_attr['replay_time'].astype(int).isin(df_inj_both_flag_a['time'].unique())]

In [19]:
df_inj_both_flag_a_attrs

,replay_time,time_H,coa_phase,eff_dist_H,approximant,chi_p,z,distance,spin2z,fref,...,Mc,lambda1,mass2,flow,logpdraw_right_ascension_declination,logpdraw_z_mass1_source_mass2_source_spin1x_spin1y_spin1z_spin2x_spin2y_spin2z,logpdraw_lambda1_lambda2_GIVEN_mass1_source_mass2_source,logpdraw_inclination_polarization_coa_phase,logpdraw_eccentricity,logpdraw_time
0,1.324070e+09,1.263782e+09,4.400568,658.213588,b'IMRPhenomPv2_NRTidalv2',0.489192,0.205563,1048.320953,-0.083213,10.0,...,12.682835,0.0,13.432107,10.0,-2.824640,-11.318613,0.0,-4.490021,0.0,-inf
50,1.324757e+09,1.264469e+09,6.106272,1376.035529,b'IMRPhenomPv2_NRTidalv2',0.165124,0.293007,1565.981419,0.268909,10.0,...,44.001881,0.0,37.466368,10.0,-2.950116,-4.082969,0.0,-4.750454,0.0,-inf
111,1.324846e+09,1.264558e+09,1.488794,1544.489371,b'IMRPhenomPv2_NRTidalv2',0.708649,0.295344,1580.334641,-0.045178,10.0,...,17.013246,0.0,13.265519,10.0,-2.868021,-13.749169,0.0,-5.237779,0.0,-inf
168,1.322665e+09,1.262377e+09,5.271821,1374.320669,b'IMRPhenomPv2_NRTidalv2',0.377101,0.290754,1552.172509,-0.141551,10.0,...,11.566187,0.0,12.518955,10.0,-2.630344,-10.701397,0.0,-5.141427,0.0,-inf
213,1.322688e+09,1.262400e+09,0.891948,3031.443021,b'IMRPhenomPv2_NRTidalv2',0.048779,0.646509,3992.662660,-0.008014,10.0,...,57.677887,0.0,61.560269,10.0,-2.693958,-2.336124,0.0,-4.318012,0.0,-inf
224,1.323209e+09,1.262921e+09,1.593250,2095.446538,b'IMRPhenomPv2_NRTidalv2',0.274955,0.529894,3140.708789,0.023824,10.0,...,75.662565,0.0,82.244019,10.0,-2.539189,-9.837387,0.0,-4.773088,0.0,-inf
253,1.322914e+09,1.262626e+09,5.121788,229.745252,b'IMRPhenomPv2_NRTidalv2',0.183640,0.078681,370.816808,0.001646,10.0,...,25.019722,0.0,27.002010,10.0,-2.540166,-6.010108,0.0,-4.168449,0.0,-inf
300,1.324978e+09,1.264690e+09,4.063662,744.723446,b'IMRPhenomPv2_NRTidalv2',0.232550,0.264845,1395.086510,0.094368,10.0,...,38.884053,0.0,40.821379,10.0,-2.746371,-4.558420,0.0,-4.750904,0.0,-inf
316,1.325582e+09,1.265294e+09,1.050888,486.222231,b'IMRPhenomPv2_NRTidalv2',0.262977,0.142148,698.119175,-0.162536,10.0,...,35.909107,0.0,40.977866,10.0,-2.798151,-9.686006,0.0,-4.051235,0.0,-inf
317,1.322692e+09,1.262404e+09,5.037607,59.415664,b'IMRPhenomPv2_NRTidalv2',0.466045,0.145648,716.863844,0.088015,10.0,...,46.864496,0.0,49.892125,10.0,-2.571345,-14.150551,0.0,-3.686603,0.0,-inf


In [20]:
df_inj_both_flag_a_attrs.to_csv('/files/attributes_inj_flagged_both_a.csv', index=None)

OSError: Cannot save file into a non-existent directory: '/files'